In [2]:
from typing import *
from ortools.sat.python.cp_model import CpModel, CpSolver, OPTIMAL, FEASIBLE

### N Queens

In [3]:
def solver(N: int) -> Optional[List[int]]:
  model = CpModel()

  # i is row number and queens[i] is column number
  queens = [model.NewIntVar(0, N-1, f'q-{i}') for i in range(N)]

  model.AddAllDifferent(queens)
  # Add constraint where all rows + cols are different
  model.AddAllDifferent(queens[i] + i for i in range(N))
  # Add all constraints where all rows - cols are different
  model.AddAllDifferent(queens[i] - i for i in range(N))

  solver = CpSolver()
  status = solver.Solve(model)

  if status == OPTIMAL or status == FEASIBLE:
    return list(map(solver.Value, queens))

solver(4)

[2, 0, 3, 1]

### Sudoku Solver

In [4]:
def solver(initial_state: List[List[int]]) -> Optional[List[List[int]]]:
  model = CpModel()

  sudoku = [[model.NewIntVar(1, 9, f's-{i}{j}') for i in range(9)] for j in range(9)]

  for i in range(9):
    for j in range(9):
      if not initial_state[i][j]:
        continue
      model.Add(sudoku[i][j] == initial_state[i][j])

  for i in range(9):
    model.AddAllDifferent(sudoku[i][j] for j in range(9))
    model.AddAllDifferent(sudoku[j][i] for j in range(9))

  for r in range(3):
    for c in range(3):
      model.AddAllDifferent(sudoku[i][j] for i in range(r*3, r*3+3) for j in range(c*3, c*3+3))

  solver = CpSolver()
  status = solver.Solve(model)

  if status == FEASIBLE or status == OPTIMAL:
    return list(map(lambda row: list(map(solver.Value, row)), sudoku))

def is_valid_sudoku(grid):
  for i in range(9):
    if len(set(grid[i])) != 9 or len(set(grid[j][i] for j in range(9))) != 9:
      return False
  for r in range(0, 9, 3):
    for c in range(0, 9, 3):
      block = [grid[r+i][c+j] for i in range(3) for j in range(3)]
      if len(set(block)) != 9:
        return False
  return True

solution = solver([
  [5, 3, 0, 0, 7, 0, 0, 0, 0],
  [6, 0, 0, 1, 9, 5, 0, 0, 0],
  [0, 9, 8, 0, 0, 0, 0, 6, 0],
  [8, 0, 0, 0, 6, 0, 0, 0, 3],
  [4, 0, 0, 8, 0, 3, 0, 0, 1],
  [7, 0, 0, 0, 2, 0, 0, 0, 6],
  [0, 6, 0, 0, 0, 0, 2, 8, 0],
  [0, 0, 0, 4, 1, 9, 0, 0, 5],
  [0, 0, 0, 0, 8, 0, 0, 7, 9]
])

solution and is_valid_sudoku(solution) and print(*solution, sep="\n")

[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]


### Magic Square

In [5]:
def solver(n: int, s: int) -> Optional[List[List[int]]]:
  model = CpModel()

  squares = [[model.NewIntVar(1, n*n, f'sq-{i}{j}') for i in range(n)] for j in range(n)]

  model.AddAllDifferent(sq for row in squares for sq in row)
  for i in range(n):
    model.Add(sum(squares[i]) == s)
    model.Add(sum(squares[j][i] for j in range(n)) == s)

  model.Add(sum(squares[i][i] for i in range(n)) == s)
  model.Add(sum(squares[i][n-1 - i] for i in range(n)) == s)

  solver = CpSolver()
  status = solver.Solve(model)

  if status == FEASIBLE or status == OPTIMAL:
    return list(map(lambda row: list(map(solver.Value, row)), squares))

solution = solver(3, 15)
solution and print(*solution, sep="\n")

[8, 1, 6]
[3, 5, 7]
[4, 9, 2]


### Graph Coloring

In [6]:
def solver(graph: Dict[int, List[int]], k: int = 3):
  if k <= 1:
    return None

  model = CpModel()

  n = len(graph)
  nodes = [model.NewIntVar(1, k, f'u-{i}') for i in range(n)]

  visited = set()
  for u in range(n):
    for v in graph[u]:
      if (u, v) in visited:
        continue
      visited.add((u, v))
      model.Add(nodes[u] != nodes[v])

  solver = CpSolver()
  status = solver.Solve(model)

  if status == OPTIMAL or status == FEASIBLE:
    return list(map(solver.Value, nodes))

solution = solver({
  0: [1, 2],
  1: [0, 2, 3],
  2: [0, 1, 3],
  3: [1, 2]
})

print(solution)

[2, 1, 3, 2]


### Knapsack

In [7]:
def solver(capacity: int, weights: List[int], values: List[int]) -> List[int]:
  if len(weights) != len(values) or capacity <= 0:
    return None

  model = CpModel()

  n = len(weights)
  items = [model.NewBoolVar(f'i-{i}') for i in range(n)]

  model.Add(sum(weights[i] * items[i] for i in range(n)) <= capacity)
  model.Maximize(sum(values[i] * items[i] for i in range(n)))

  solver = CpSolver()
  status = solver.Solve(model)

  if status == FEASIBLE or status == OPTIMAL:
    return list(map(solver.Value, items))

solution = solver(
  15,
  [4, 5, 6, 2, 3],
  [10, 20, 30, 5, 12]
)

print(solution)

[0, 1, 1, 0, 1]


### Job Scheduling

In [8]:
Task = Tuple[int, int]

def solver(jobs: List[List[Task]]) -> List[List[int]]:
  model = CpModel()

  n_jobs = len(jobs) # num jobs
  n_machines = 0 # num machines
  max_end = 0
  
  tasks = {}
  # extract all tasks
  for i, job in enumerate(jobs):
    for j, (machine, unit) in enumerate(job):
      tasks[(i, j)] = (machine, unit)
      n_machines = max(n_machines, machine + 1) # finds total machines
      max_end += unit
  
  starts = {}
  ends = {}
  intervals = {}
  for i, j in tasks:
    machine, unit = tasks[(i, j)]
    name = f'i{i}j{j}'
    starts[(i, j)] = model.NewIntVar(0, max_end, f's_{name}')
    ends[(i, j)] = model.NewIntVar(0, max_end, f'e_{name}')
    intervals[(i, j)] = model.NewIntervalVar(starts[(i, j)], unit, ends[(i, j)], f'i_{name}')

  # this prevents task ordering from changing
  for i in range(n_jobs):
    for j in range(len(jobs[i]) - 1):
      model.Add(ends[(i, j)] <= starts[(i, j+1)])

  # keeping track of all intervals that occur for a machine
  # i.e. the intervals of tasks on machines
  machine_intervals = {machine: [] for machine in range(n_machines)}
  for i, j in tasks:
    machine, _ = tasks[(i, j)]
    machine_intervals[machine].append(intervals[(i, j)])
  
  # ensuring non of the intervals for a single machine overlap
  for machine in range(n_machines):
    model.AddNoOverlap(machine_intervals[machine])

  makespan = model.NewIntVar(0, max_end, 'makespan')
  # force makespan to be the last of end times
  # i.e. the value makespan will take will always be the end time for the last job
  model.AddMaxEquality(makespan, (ends[(i, len(jobs[i])-1)] for i in range(n_jobs)))
  # we want to minimize makespan to get the shortets scheduling
  model.Minimize(makespan)

  solver = CpSolver()
  status = solver.Solve(model)

  if status == OPTIMAL or status == FEASIBLE:
    schedule = []
    for i in range(n_jobs):
      job_schedule = []
      for j in range(len(jobs[i])):
        start = solver.Value(starts[(i, j)])
        end = solver.Value(ends[(i, j)])
        machine, _ = tasks[(i, j)]
        job_schedule.append((machine, start, end))
      schedule.append(job_schedule)
    return schedule, solver.ObjectiveValue() # same as solver.Value(makespan)

  return None, None

  
schedule, makespan = solver([
  [  # Job 0: (machine, duration) for each task
    (0, 3),  # Task 0: Runs on M0 for 3 units
    (1, 2),  # Task 1: Runs on M1 for 2 units
    (2, 2),  # Task 2: Runs on M2 for 2 units
  ],
  [  # Job 1
    (0, 2),
    (2, 1),
    (1, 4),
  ],
  [  # Job 2
    (1, 4),
    (0, 3),
    (2, 1),
  ],
])

print('span:', makespan)
for i in range(3):
  print(f'job {i}')
  for j in range(3):
    machine, start, end = schedule[i][j]
    print(f'  machine {machine}:', f'{start}-{end}')

span: 10.0
job 0
  machine 0: 0-3
  machine 1: 4-6
  machine 2: 6-8
job 1
  machine 0: 3-5
  machine 2: 5-6
  machine 1: 6-10
job 2
  machine 1: 0-4
  machine 0: 5-8
  machine 2: 8-9


### TSP

In [14]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def solver(distance_matrix):
  manager = pywrapcp.RoutingIndexManager(len(distance_matrix), 1, 0)
  
  routing = pywrapcp.RoutingModel(manager)
  
  def distance_cb(from_i, to_i):
    return distance_matrix[manager.IndexToNode(from_i)][manager.IndexToNode(to_i)]
  
  transit_callback_index = routing.RegisterTransitCallback(distance_cb)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
  
  search_params = pywrapcp.DefaultRoutingSearchParameters()
  search_params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
  
  solution = routing.SolveWithParameters(search_params)
  
  if solution:
    route = []
    index = routing.Start(0)
    while not routing.IsEnd(index):
      route.append(manager.IndexToNode(index))
      index = solution.Value(routing.NextVar(index))
    route.append(manager.IndexToNode(index))
    total_distance = solution.ObjectiveValue()
    return route, total_distance
  return None, None

route, distance = solver([
  [0, 10, 15, 20, 25],    # From City 0 to others
  [10, 0, 35, 25, 30],    # From City 1 to others
  [15, 35, 0, 30, 40],    # From City 2 to others
  [20, 25, 30, 0, 50],    # From City 3 to others
  [25, 30, 40, 50, 0],    # From City 4 to others
])

print(distance)
print(route)

125
[0, 2, 3, 1, 4, 0]
